# Point Process

In [2]:
import pandas as pd
import numpy as np
#import theano
import pymc as pm
from scipy.stats import ks_2samp
import seaborn as sns
import matplotlib.pyplot as plt
import glob

sns.set_context('notebook')
plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))
#theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
import warnings

Running on PyMC3 v5.9.2


/var/folders/d6/n7zzsskd0jv11v_qxbdvkkf40000gn/T/ipykernel_20946/3338226896.py:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


## Dirichlet Mixture Model

In [12]:
def Bayesian_mixture_model_old(ISI_data):
    
    with pm.Model() as model:
        
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.2)
        sigma = pm.Uniform('sigma',lower=0.0001,upper=0.5)
        obs2 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu2,sigma=sigma)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu3,sigma=sigma3)


        w = pm.Dirichlet('w', a=np.array([1., .4, .4]))

        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)


        #step = pm.Metropolis(target_accept=0.9)
        #trace = pm.sample(step=step,draws=500,tune=100,progressbar=False,random_seed=10,cores=6)
     

    map_estimate = pm.find_MAP(model=model)
    del map_estimate['w_stickbreaking__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]
    
    del map_estimate['w']
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    #with model:
    #    ppc_trace = pm.sample_posterior_predictive(trace, 50,model=model,progressbar=False,random_seed=10)
    
    #lista_samples=[]
    #for i in list(ppc_trace['like']):
    #    lista_samples.extend(i)
    
    
    #print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

In [13]:
def Bayesian_mixture_model(ISI_data):
    with pm.Model() as model:
        ##### WALD DISTRIBUTION (INVERSE GAUSSIAN)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.2)
        sigma2 = pm.Uniform('sigma2',lower=0.0001,upper=0.5)
        obs2 = pm.TruncatedNormal.dist(mu=mu2, sigma=sigma2, lower=0.0)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.TruncatedNormal.dist(mu=mu3, sigma=sigma3, lower=0.0)


        w = pm.Dirichlet('w', a=np.array([1., .4, .4]))
        #w = pm.Dirichlet('w', a=np.array([1., .4]))

        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)
        #like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2], observed=ISI_data)

        step = pm.NUTS(target_accept=0.9)
        trace = pm.sample(step=step,draws=1000,chains=1,tune=1000,cores=4)
        
        #ppc_trace = pm.sample_posterior_predictive(trace,model=model)
        
    map_estimate = pm.find_MAP(model=model)
    
    del map_estimate['w_simplex__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma2_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]

    del map_estimate['w']


    return map_estimate#, ppc_trace


## Inverse Gaussian

In [4]:
def Inverse_Gaussian_model(ISI_data):    
    
    with pm.Model() as model:
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald('like',mu=mu1,lam=lam1,observed=ISI_healthy)


        step = pm.NUTS(target_accept=0.9)
        trace = pm.sample(step=step,draws=1000,tune=1000,cores=6)

        map_estimate = pm.find_MAP(model=model)
        
    
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    
    
    
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    with model:
        ppc_trace = pm.sample_posterior_predictive(trace, 200,model=model)
    
    lista_samples=[]
    for i in list(ppc_trace['like']):
        lista_samples.extend(i)
    
    
    print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

## Import all neural net

In [14]:
list_dir=glob.glob("Data after SS/*.txt")

In [15]:
list_dir_ok = []
for net in list_dir:
    #if 'After' not in net:
    print(net,'ok')
    list_dir_ok.append(net)

Data after SS/After2019-01-23T15-49-43MEA2 healthy cortical .h5.txt.txt ok
Data after SS/After2019-01-23T16-06-32Pop1 healthy cortical .h5.txt.txt ok
Data after SS/After2018-11-27T10-56-39MiP5 KA stimulation.h5.txt.txt ok
Data after SS/After2018-11-27T10-29-42POP 2 BL .h5.txt.txt ok
Data after SS/After2019-01-23T11-05-52MIP3 health cortical .h5.txt.txt ok
Data after SS/After2018-11-27T11-24-28MiP3 stimulation.h5.txt.txt ok
Data after SS/After2018-11-27T10-40-53POP 3 BL .h5.txt.txt ok
Data after SS/After2019-01-23T16-22-47Pop3 healthy cortical .h5.txt.txt ok
Data after SS/After2018-11-27T12-03-02POP 2 stimulation.h5.txt.txt ok


In [16]:
n_neurons_healthy = 0
n_neurons_lrrk2 = 0
for net in list_dir_ok:
    list_neurons = np.genfromtxt(net, delimiter=',')
    if 'health' in net:
            print('H',net,list_neurons.shape[0])
            n_neurons_healthy = n_neurons_healthy+list_neurons.shape[0]
            
    elif 'Healthy' in net:
            print('H',net,list_neurons.shape[0])
            n_neurons_healthy = n_neurons_healthy+list_neurons.shape[0]
    else:
            print('L',net,list_neurons.shape[0])
            n_neurons_lrrk2 = n_neurons_lrrk2+list_neurons.shape[0]
print('Total healthy neurons: ',n_neurons_healthy,'\n')
print('Total lrrk2 neurons: ',n_neurons_lrrk2,'\n')


H Data after SS/After2019-01-23T15-49-43MEA2 healthy cortical .h5.txt.txt 91
H Data after SS/After2019-01-23T16-06-32Pop1 healthy cortical .h5.txt.txt 83
L Data after SS/After2018-11-27T10-56-39MiP5 KA stimulation.h5.txt.txt 121
L Data after SS/After2018-11-27T10-29-42POP 2 BL .h5.txt.txt 94
H Data after SS/After2019-01-23T11-05-52MIP3 health cortical .h5.txt.txt 91
L Data after SS/After2018-11-27T11-24-28MiP3 stimulation.h5.txt.txt 106
L Data after SS/After2018-11-27T10-40-53POP 3 BL .h5.txt.txt 97
H Data after SS/After2019-01-23T16-22-47Pop3 healthy cortical .h5.txt.txt 96
L Data after SS/After2018-11-27T12-03-02POP 2 stimulation.h5.txt.txt 80
Total healthy neurons:  361 

Total lrrk2 neurons:  498 



## Run the model on baseline in the interval 100-300s

In [22]:
dataframe = pd.DataFrame()
counter_net=1
counter=0
target = 0
for net in list_dir_ok:
    print(counter_net,') ',net)
    counter_net+=1
    list_neurons = np.genfromtxt(net, delimiter=',')
    counter=0
    print('Original number of neurons: ',len(list_neurons))
    for neuron in list_neurons:
        neuron=neuron[neuron>0*10000]
        neuron=neuron[neuron<200*10000]
        print('  Neuron with ',neuron.shape[0],'spikes')
        if neuron.shape[0]>800:
            
            counter+=1
        else:
            print('    Excluded neuron with n spikes = ',neuron.shape[0])
            continue
        
        ISI_healthy = np.diff(neuron)/10000
        
            
        map_estimate = Bayesian_mixture_model(ISI_healthy)
        
        if 'Healthy' in net:
            print('target Healthy')
            map_estimate['Target']=0
            
        elif 'healthy' in net:
            print('target healthy')
            map_estimate['Target']=0
        elif 'health' in net:
            print('target health')
            map_estimate['Target']=0

        else:
            map_estimate['Target']=1
        
        if 'KA' in net:
            map_estimate['Stimulation']=1
            
        elif 'stimulation' in net:
            map_estimate['Stimulation']=1
        else:
            map_estimate['Stimulation']=0
            
        df = pd.DataFrame.from_dict(map_estimate,orient='index')
        dataframe = pd.concat([dataframe,df],axis = 1)
    print('Final number of neurons: ',counter)
    print('Target = ',target)

1 )  Data after SS/After2019-01-23T15-49-43MEA2 healthy cortical .h5.txt.txt
Original number of neurons:  91
  Neuron with  323 spikes
    Excluded neuron with n spikes =  323
  Neuron with  319 spikes
    Excluded neuron with n spikes =  319
  Neuron with  323 spikes
    Excluded neuron with n spikes =  323
  Neuron with  440 spikes
    Excluded neuron with n spikes =  440
  Neuron with  295 spikes
    Excluded neuron with n spikes =  295
  Neuron with  344 spikes
    Excluded neuron with n spikes =  344
  Neuron with  481 spikes
    Excluded neuron with n spikes =  481
  Neuron with  571 spikes
    Excluded neuron with n spikes =  571
  Neuron with  370 spikes
    Excluded neuron with n spikes =  370
  Neuron with  419 spikes
    Excluded neuron with n spikes =  419
  Neuron with  294 spikes
    Excluded neuron with n spikes =  294
  Neuron with  176 spikes
    Excluded neuron with n spikes =  176
  Neuron with  427 spikes
    Excluded neuron with n spikes =  427
  Neuron with  423 s

Sequential sampling (1 chains in 1 job)
NUTS: [mu1, lam1, mu2, sigma2, mu3, sigma3, w]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 95 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks



target healthy
  Neuron with  308 spikes
    Excluded neuron with n spikes =  308
  Neuron with  239 spikes
    Excluded neuron with n spikes =  239
  Neuron with  304 spikes
    Excluded neuron with n spikes =  304
  Neuron with  461 spikes
    Excluded neuron with n spikes =  461
  Neuron with  438 spikes
    Excluded neuron with n spikes =  438
  Neuron with  392 spikes
    Excluded neuron with n spikes =  392
  Neuron with  584 spikes
    Excluded neuron with n spikes =  584
  Neuron with  458 spikes
    Excluded neuron with n spikes =  458
  Neuron with  308 spikes
    Excluded neuron with n spikes =  308
  Neuron with  239 spikes
    Excluded neuron with n spikes =  239
  Neuron with  304 spikes
    Excluded neuron with n spikes =  304
  Neuron with  461 spikes
    Excluded neuron with n spikes =  461
  Neuron with  438 spikes
    Excluded neuron with n spikes =  438
  Neuron with  392 spikes
    Excluded neuron with n spikes =  392
  Neuron with  584 spikes
    Excluded neuron 

Sequential sampling (1 chains in 1 job)
NUTS: [mu1, lam1, mu2, sigma2, mu3, sigma3, w]


...

Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 59 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks



  Neuron with  597 spikes
    Excluded neuron with n spikes =  597
  Neuron with  740 spikes
    Excluded neuron with n spikes =  740
  Neuron with  369 spikes
    Excluded neuron with n spikes =  369
  Neuron with  445 spikes
    Excluded neuron with n spikes =  445
  Neuron with  312 spikes
    Excluded neuron with n spikes =  312
  Neuron with  983 spikes


Sequential sampling (1 chains in 1 job)
NUTS: [mu1, lam1, mu2, sigma2, mu3, sigma3, w]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 62 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks



  Neuron with  597 spikes
    Excluded neuron with n spikes =  597
  Neuron with  740 spikes
    Excluded neuron with n spikes =  740
  Neuron with  369 spikes
    Excluded neuron with n spikes =  369
  Neuron with  445 spikes
    Excluded neuron with n spikes =  445
  Neuron with  312 spikes
    Excluded neuron with n spikes =  312
Final number of neurons:  10
Target =  0


In [23]:
final = dataframe.T
final.to_csv('Data after PP/DataAfter')